In [1]:
# The objective of this competition is to use historical transaction data to predict the riskability of an new transactio

In [1]:
import numpy as np
import pandas as pd 
import os
import warnings
warnings.filterwarnings('ignore')
# import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
# List files available
print(os.listdir("../sourcedata/"))

['atec_anti_fraud_test_b.csv', 'atec_anti_fraud_train.csv']


In [3]:
train_file = "../sourcedata/atec_anti_fraud_train.csv"
test_file = "../sourcedata/atec_anti_fraud_test_b.csv"

df_train = pd.read_csv(train_file,header=0,index_col=None)
df_test = pd.read_csv(test_file,header=0,index_col=None)

In [4]:
print("training data shape:", df_train.shape)
print("testing data shape:", df_test.shape)

training data shape: (994731, 300)
testing data shape: (500538, 299)


In [5]:
df_train.head()

,id,label,date,f1,f2,f3,f4,f5,f6,f7,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,0,20171103,0,0,0,0,100807.0,0,5,...,301.0,312.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0
1,d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...,0,20170917,0,1,1,1,100805.0,1,5,...,302.0,324.0,391.0,13.0,302.0,160.0,160.0,161.0,8.0,160.0
2,1270cb8a85eedd57672b2c6297fa5633e36773a2c3a351...,0,20171022,0,0,1,0,100102.0,0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9fa009724ee7ff9d688ae321304fbc78f608cdabbfdd2b...,0,20171029,0,0,0,1,100807.0,1,4,...,302.0,322.0,341.0,57.0,251.0,175.0,176.0,176.0,49.0,150.0
4,1da482485d7e8bcefae7e9d0d1167cec3ac111cfa71d8b...,0,20171002,1,1,0,1,100805.0,1,5,...,302.0,301.0,301.0,74.0,302.0,182.0,181.0,182.0,51.0,181.0


In [6]:
for col in ["f"+str(j) for j in range(1,20) if  j!=5]:
    df_train[col] = df_train[col].astype(np.uint8)
    df_test[col] = df_test[col].astype(np.uint8)
#     df_train.loc[df_train[col],col].astype(np.uint8)
#     df_test.loc[df_test[col],col].astype(np.uint8)
# df_train.loc[df_train["label"],"label"].astype(np.int8)

In [7]:
# df_train["id"] = df_train["id"].astype(str)
# df_train.loc[df_train["label"],"label"].astype(np.int8)
# df_train["date"] = df_train["date"].astype(np.uint32)
# df_test["id"] = df_test["id"].astype(str)
# df_test["date"] = df_test["date"].astype(np.uint32)

In [8]:
# df_train.drop_duplicates(inplace=True)
print("training data shape:", df_train.shape)

training data shape: (994731, 300)


In [9]:
print(df_train["id"].nunique())
print(df_test["id"].nunique())

994731
500538


In [10]:
intersect_ = set(df_test["id"].tolist()).intersection(df_train["id"].tolist())
print(len(intersect_))

0


In [11]:
df_train["label"].value_counts()

 0    977884
 1     12122
-1      4725
Name: label, dtype: int64

In [6]:
# version one: filter data with label -1
# df_train = df_train.loc[df_train["label"]>=0]
# version two: 
df_train.loc[df_train["label"]==-1,"label"] = 1

In [7]:
import gc
gc.collect()
df_train["label"].value_counts()

0    977884
1     16847
Name: label, dtype: int64

In [8]:
df_train["label"] = df_train["label"].astype(np.uint8)
df_train_id = df_train[["id","date","label"]]
df_train.drop(labels=["id","date","label"],axis=1,inplace=True)
df_test_id = df_test[["id","date"]]
df_test.drop(labels=["id","date"],axis=1,inplace=True)
gc.collect()

7

In [9]:
anormaly_cols = []
f_cols = df_train.columns
for col in f_cols:
#     print(col)
    mean_train = df_train[col].mean()
    std_train = df_train[col].std()
    mean_test = df_test[col].mean()
    std_test = df_test[col].std()
    mean_diff = max(mean_train/mean_test,mean_test/mean_train)
    std_diff = max(std_train/std_test,std_test/std_train)
    if mean_diff>5 and  std_diff>5:
        anormaly_cols.append(col)
print(anormaly_cols)
print(len(anormaly_cols))

['f20', 'f21', 'f22', 'f23', 'f32', 'f33', 'f34', 'f35', 'f48', 'f49', 'f50', 'f51', 'f64', 'f65', 'f67', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137', 'f138', 'f139', 'f140', 'f141', 'f142', 'f143', 'f144', 'f145', 'f146', 'f147', 'f148', 'f149', 'f150', 'f151', 'f152', 'f153', 'f161', 'f162', 'f163', 'f164', 'f165']
63


In [10]:
df_train.drop(labels=anormaly_cols,axis=1,inplace=True)
df_test.drop(labels=anormaly_cols,axis=1,inplace=True)
gc.collect()
print("training data shape:", df_train.shape)
print("testing data shape:", df_test.shape)

training data shape: (994731, 234)
testing data shape: (500538, 234)


In [11]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [12]:
# Missing values statistics
missing_values_train = missing_values_table(df_train)
missing_values_test = missing_values_table(df_test)

Your selected dataframe has 234 columns.
There are 216 columns that have missing values.
Your selected dataframe has 234 columns.
There are 216 columns that have missing values.


In [13]:
# print(missing_values_train.head(40))
# print(missing_values_test.head(60))
# missing_values_train.to_csv("../statsfile/stats.csv",mode="a")
# missing_values_test.to_csv("../statsfile/stats.csv",mode="a")
print(missing_values_test.head())

     Missing Values  % of Total Values
f46          466775               93.3
f43          466775               93.3
f47          466775               93.3
f45          466775               93.3
f44          466775               93.3


In [14]:
# Number of each type of column
df_train.dtypes.value_counts()

float64    216
int64       18
dtype: int64

In [15]:
print(missing_values_train.index)

Index(['f37', 'f38', 'f46', 'f45', 'f44', 'f43', 'f42', 'f41', 'f40', 'f39',
       ...
       'f230', 'f229', 'f228', 'f227', 'f226', 'f225', 'f224', 'f223', 'f222',
       'f242'],
      dtype='object', length=216)


In [16]:
fourty_percent_na_cols_train = missing_values_train[missing_values_train["% of Total Values"]>40].index.tolist() 
print(len(fourty_percent_na_cols_train))

12


In [17]:
fourty_percent_na_cols_test = missing_values_test[missing_values_test["% of Total Values"]>40].index.tolist() 
print(len(fourty_percent_na_cols_test))

73


In [18]:
fourty_percent_na_cols = list(set(fourty_percent_na_cols_test+fourty_percent_na_cols_train))
print(len(fourty_percent_na_cols))

73


In [20]:
df_train.drop(labels=fourty_percent_na_cols,axis=1,inplace=True)
df_test.drop(labels=fourty_percent_na_cols,axis=1,inplace=True)

In [21]:
gc.collect()
print("training data shape:", df_train.shape)
print("testing data shape:", df_test.shape)

training data shape: (994731, 161)
testing data shape: (500538, 161)


In [22]:
df_train.drop_duplicates(inplace=True)
print("training data shape:", df_train.shape)

training data shape: (994731, 161)


In [23]:
missing_values_train = missing_values_table(df_train)
missing_values_test = missing_values_table(df_test)


Your selected dataframe has 161 columns.
There are 143 columns that have missing values.
Your selected dataframe has 161 columns.
There are 143 columns that have missing values.


In [24]:
print(missing_values_train.head(10))
print(missing_values_test.head(10))

      Missing Values  % of Total Values
f31           211036               21.2
f28           211036               21.2
f29           211036               21.2
f30           211036               21.2
f283          207585               20.9
f288          207585               20.9
f287          207585               20.9
f286          207585               20.9
f285          207585               20.9
f284          207585               20.9
      Missing Values  % of Total Values
f31           134096               26.8
f28           134096               26.8
f29           134096               26.8
f30           134096               26.8
f283          132844               26.5
f288          132844               26.5
f287          132844               26.5
f286          132844               26.5
f285          132844               26.5
f284          132844               26.5


In [30]:
# df_train.to_csv("../transdata/train_drop_v1.csv",header=True,index=False)
# df_test.to_csv("../transdata/test_drop_v1.csv",header=True,index=False)

In [25]:
ds1 = df_train.describe()
print(ds1)
ds1.to_csv("../statsfile/stats1.csv",mode="a")
ds2 = df_test.describe()
print(ds2)
ds2.to_csv("../statsfile/stats1.csv",mode="a")

                  f1             f2             f3             f4  \
count  994731.000000  994731.000000  994731.000000  994731.000000   
mean        0.726283       0.500948       0.699285       0.525461   
std         0.651947       0.500363       0.640534       0.523872   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         1.000000       1.000000       1.000000       1.000000   
75%         1.000000       1.000000       1.000000       1.000000   
max         2.000000       2.000000       2.000000       2.000000   

                  f5             f6             f7             f8  \
count  794906.000000  994731.000000  994731.000000  994731.000000   
mean   100652.575330       0.701303       4.201802       1.404783   
std      4023.235737       0.757799       1.922035       0.579449   
min     10000.000000       0.000000       0.000000       0.000000   
25%    100802.000000       0.0000

In [33]:
# print(missing_values_train)

In [26]:
ten2fifty_percent_na_cols_train = missing_values_train[missing_values_train["% of Total Values"]>6].index.tolist() 
print(len(ten2fifty_percent_na_cols_train))
zero2ten_percent_na_cols_train = missing_values_train[missing_values_train["% of Total Values"]<=6].index.tolist() 
print(len(zero2ten_percent_na_cols_train))

100
43


In [27]:
ten2fifty_percent_na_cols_test = missing_values_test[missing_values_test["% of Total Values"]>6].index.tolist() 
print(len(ten2fifty_percent_na_cols_test))
zero2ten_percent_na_cols_test = missing_values_test[missing_values_test["% of Total Values"]<=6].index.tolist() 
print(len(zero2ten_percent_na_cols_test))

100
43


In [28]:
ten2fifty_percent_na_cols = list(set(ten2fifty_percent_na_cols_train+ten2fifty_percent_na_cols_test))
print(len(ten2fifty_percent_na_cols))
zero2ten_percent_na_cols = list(set(zero2ten_percent_na_cols_train+zero2ten_percent_na_cols_test))
print(len(zero2ten_percent_na_cols))

100
43


In [29]:
# Create an nan flag column
ten2fifty_percent_na_cols_nan = []
for col in ten2fifty_percent_na_cols:
    col_name = col+"_nan"
    ten2fifty_percent_na_cols_nan.append(col_name)
    df_train[col_name] = 0
    df_train.loc[df_train[col].isnull(),col_name] = 1
    df_test[col_name] = 0
    df_test.loc[df_test[col].isnull(),col_name] = 1
    df_train[col_name] = df_train[col_name].astype(np.uint8)
    df_test[col_name] = df_test[col_name].astype(np.uint8)
#     df_train.loc[df_train[col_name],col_name].astype(np.uint8)
#     df_test.loc[df_test[col_name],col_name].astype(np.uint8)

In [30]:
ten2fifty_percent_cols_na = ten2fifty_percent_na_cols+ten2fifty_percent_na_cols_nan

In [31]:
df_ten2fifty_percent_na_cols_train = df_train[ten2fifty_percent_cols_na]
df_train.drop(labels=ten2fifty_percent_cols_na,axis=1,inplace=True)

In [32]:
df_ten2fifty_percent_na_cols_test = df_test[ten2fifty_percent_cols_na]
df_test.drop(labels=ten2fifty_percent_cols_na,axis=1,inplace=True)

In [33]:
print("training data shape:", df_train.shape)
print("testing data shape:", df_test.shape)

training data shape: (994731, 61)
testing data shape: (500538, 61)


In [34]:
mice_cols = df_train.columns
print(mice_cols)


Index(['f1', 'f2', 'f3', 'f4', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12',
       'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f211', 'f212', 'f213',
       'f214', 'f215', 'f216', 'f217', 'f218', 'f219', 'f220', 'f221', 'f222',
       'f223', 'f224', 'f225', 'f226', 'f227', 'f228', 'f229', 'f230', 'f231',
       'f232', 'f233', 'f234', 'f235', 'f236', 'f237', 'f238', 'f239', 'f240',
       'f241', 'f242', 'f243', 'f244', 'f245', 'f246', 'f247', 'f248', 'f249',
       'f250', 'f251', 'f252', 'f253'],
      dtype='object')


In [35]:
df_train = pd.concat([df_train_id,df_train,df_ten2fifty_percent_na_cols_train],axis=1)
df_test = pd.concat([df_test_id,df_test,df_ten2fifty_percent_na_cols_test],axis=1)

In [36]:
del df_train_id,df_ten2fifty_percent_na_cols_train,df_test_id,df_ten2fifty_percent_na_cols_test
gc.collect()

385

In [37]:
print("training data shape:", df_train.shape)
print("testing data shape:", df_test.shape)

training data shape: (994731, 264)
testing data shape: (500538, 263)


In [38]:
ds1 = df_train.describe()
print(ds1)
ds1.to_csv("../statsfile/stats1.csv",mode="a")
ds2 = df_test.describe()
print(ds2)
ds2.to_csv("../statsfile/stats1.csv",mode="a")

               date          label             f1             f2  \
count  9.947310e+05  994731.000000  994731.000000  994731.000000   
mean   2.017098e+07       0.016936       0.726283       0.500948   
std    6.011735e+01       0.129033       0.651947       0.500363   
min    2.017090e+07       0.000000       0.000000       0.000000   
25%    2.017092e+07       0.000000       0.000000       0.000000   
50%    2.017101e+07       0.000000       1.000000       1.000000   
75%    2.017102e+07       0.000000       1.000000       1.000000   
max    2.017110e+07       1.000000       2.000000       2.000000   

                  f3             f4             f6             f7  \
count  994731.000000  994731.000000  994731.000000  994731.000000   
mean        0.699285       0.525461       0.701303       4.201802   
std         0.640534       0.523872       0.757799       1.922035   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       

In [39]:
df_train[zero2ten_percent_na_cols] = df_train[zero2ten_percent_na_cols].fillna(0)
df_test[zero2ten_percent_na_cols] = df_test[zero2ten_percent_na_cols].fillna(0)

In [40]:
df_train.to_csv("../transdata/train_impute_v2.csv",header=True,index=False)
df_test.to_csv("../transdata/test_impute_v2.csv",header=True,index=False)


In [55]:
ds1 = df_train.describe()
print(ds1)
ds1.to_csv("../statsfile/stats1.csv",mode="a")
ds2 = df_test.describe()
print(ds2)
ds2.to_csv("../statsfile/stats1.csv",mode="a")

               date          label             f1             f2  \
count  9.900060e+05  990006.000000  990006.000000  990006.000000   
mean   2.017098e+07       0.012244       0.725547       0.500877   
std    6.012682e+01       0.109975       0.651604       0.500365   
min    2.017090e+07       0.000000       0.000000       0.000000   
25%    2.017092e+07       0.000000       0.000000       0.000000   
50%    2.017101e+07       0.000000       1.000000       1.000000   
75%    2.017102e+07       0.000000       1.000000       1.000000   
max    2.017110e+07       1.000000       2.000000       2.000000   

                  f3             f4             f6             f7  \
count  990006.000000  990006.000000  990006.000000  990006.000000   
mean        0.698765       0.524773       0.698620       4.206560   
std         0.640286       0.523246       0.753708       1.921513   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       

In [56]:
# Find correlations with the target and sort
correlations = df_train.corr()['label'].sort_values()
# Display correlations
print('Most Positive Correlations:\n', correlations.tail(20))
print('\nMost Negative Correlations:\n', correlations.head(20))

Most Positive Correlations:
 f264_nan    0.097721
f190_nan    0.097721
f177_nan    0.097721
f184_nan    0.097721
f197_nan    0.097721
f179_nan    0.097721
f6          0.126411
f264        0.149677
f28         0.162565
f29         0.165072
f261        0.166892
f260        0.181916
f271        0.185304
f53         0.195208
f259        0.197449
f52         0.205588
f270        0.215005
f30         0.240223
f31         0.242512
label       1.000000
Name: label, dtype: float64

Most Negative Correlations:
 f7     -0.098654
f12    -0.091472
f14    -0.076725
f18    -0.065231
f17    -0.064874
f185   -0.061517
f16    -0.056067
f184   -0.055365
f15    -0.054209
f183   -0.045445
f210   -0.040381
f13    -0.031939
f182   -0.029063
f209   -0.026573
f19    -0.024366
f178   -0.023836
f177   -0.017487
f176   -0.010553
f208   -0.010066
f285   -0.009610
Name: label, dtype: float64


In [57]:
correlations.to_csv("../statsfile/stats_corr.csv",mode="a")

In [58]:
f_cols = list(set(set(df_train.columns)-set(["id","label","date"]))-set(ten2fifty_percent_na_cols))
print(len(f_cols))

161


In [63]:
# Make a new dataframe for polynomial features,
poly_features = df_train[['f7', 'f12', 'f14', 'f18','f17', 'f31','f30','f270','f52','f259','label']]
poly_features_test = df_test[['f7', 'f12', 'f14', 'f18','f17', 'f31','f30','f270','f52','f259']]


In [64]:
# imputer for handling missing values
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')

In [65]:
poly_target = poly_features['label']

poly_features = poly_features.drop(columns = ['label'])

In [66]:
# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)

In [67]:
from sklearn.preprocessing import PolynomialFeatures
                                  
# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)

In [68]:
# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

Polynomial Features shape:  (990006, 286)


In [69]:
poly_transformer.get_feature_names(input_features = ['f7', 'f12', 'f14', 'f18','f17', 'f31','f30','f270','f52','f259'])[:15]

['1',
 'f7',
 'f12',
 'f14',
 'f18',
 'f17',
 'f31',
 'f30',
 'f270',
 'f52',
 'f259',
 'f7^2',
 'f7 f12',
 'f7 f14',
 'f7 f18']

In [70]:
# Create a dataframe of the features 
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(['f7', 'f12', 'f14', 'f18','f17', 'f31','f30','f270','f52','f259']))

In [ ]:
# Create a dataframe of the features 
poly_features_test = pd.DataFrame(poly_features_test, 
                             columns = poly_transformer.get_feature_names(['f7', 'f12', 'f14', 'f18','f17', 'f31','f30','f270','f52','f259']))

In [71]:
# Add in the target
poly_features['label'] = poly_target

In [80]:
poly_features.to_csv("../transdata/poly_features_train.csv",header=True,index=False)


AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [81]:
# Create a dataframe of the features 
poly_features_test = pd.DataFrame(poly_features_test, 
                             columns = poly_transformer.get_feature_names(['f7', 'f12', 'f14', 'f18','f17', 'f31','f30','f270','f52','f259']))

In [82]:
poly_features_test.to_csv("../transdata/poly_features_test.csv",header=True,index=False)

In [83]:
del df_train, df_test
gc.collect()

148

In [85]:
# Find the correlations with the target
poly_corrs = poly_features.corr()['label'].sort_values()

In [86]:
# Display most negative and most positive
print(poly_corrs.head(10))
print(poly_corrs.tail(5))

f18^2 f52      -0.002269
f12^2 f52      -0.002243
f18 f17 f52    -0.002221
f18 f52        -0.002212
f12 f18 f52    -0.002075
f7 f18 f52     -0.002051
f12 f52        -0.002021
f17 f52        -0.001859
f7 f17 f52     -0.001802
f12 f52 f259   -0.001740
Name: label, dtype: float64
f12 f14 f17    0.001879
f12^2 f17      0.001906
f12 f17^2      0.002002
label          1.000000
1                   NaN
Name: label, dtype: float64


In [79]:
gc.collect()

108